In [143]:
import pandas as pd
import numpy as np
import pyodbc

In [144]:
sales = pd.read_csv("sales_data.csv")

In [145]:
sales.head()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall
0,I138884,C241288,Clothing,5.0,1500.40,5/8/2022,Kanyon
1,I317333,C111565,Shoes,3.0,1800.51,12/12/2021,Forum Istanbul
2,I127801,C266599,Clothing,1.0,300.08,9/11/2021,Metrocity
3,I173702,C988172,Shoes,5.0,3000.85,16-05-2021,Metropol AVM
4,I337046,C189076,Books,4.0,60.60,24-10-2021,Kanyon


In [146]:
sales.tail()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall
99452,I219422,C441542,Souvenir,5.0,58.65,21-09-2022,Kanyon
99453,I325143,C569580,Food & Beverage,2.0,10.46,22-09-2021,Forum Istanbul
99454,I824010,C103292,Food & Beverage,2.0,10.46,28-03-2021,Metrocity
99455,I702964,C800631,Technology,4.0,4200.00,16-03-2021,Istinye Park
99456,I232867,C273973,Souvenir,3.0,35.19,15-10-2022,Mall of Istanbul


In [147]:
sales.shape

(99457, 7)

In [148]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   invoice_no     99457 non-null  object 
 1   customer_id    99457 non-null  object 
 2   category       99457 non-null  object 
 3   quantity       99452 non-null  float64
 4   price          99454 non-null  float64
 5   invoice_date   99457 non-null  object 
 6   shopping_mall  99453 non-null  object 
dtypes: float64(2), object(5)
memory usage: 5.3+ MB


In [149]:
sales.describe()

,quantity,price
count,99452.000000,99454.000000
mean,3.003358,689.269636
std,1.413000,941.195553
min,1.000000,5.230000
25%,2.000000,45.450000
50%,3.000000,203.300000
75%,4.000000,1200.320000
max,5.000000,5250.000000


In [150]:
sales.dtypes

invoice_no        object
customer_id       object
category          object
quantity         float64
price            float64
invoice_date      object
shopping_mall     object
dtype: object

In [151]:
# Ensuring all values in 'invoice_date' are treated as strings
sales['invoice_date'] = sales['invoice_date'].astype(str)

# Replacing '-' with '/'
sales['invoice_date'] = sales['invoice_date'].str.replace('-', '/')

# Convert the 'invoice_date' column to datetime format
sales['invoice_date'] = pd.to_datetime(sales['invoice_date'], dayfirst=True, errors='coerce')

In [152]:
sales.dtypes

invoice_no               object
customer_id              object
category                 object
quantity                float64
price                   float64
invoice_date     datetime64[ns]
shopping_mall            object
dtype: object

In [153]:
sales.isnull().sum()

invoice_no       0
customer_id      0
category         0
quantity         5
price            3
invoice_date     0
shopping_mall    4
dtype: int64

In [154]:
# Creating a function to clean date strings
def clean_date(date_str):
    if isinstance(date_str, str):
        # Replace '/' with '-'
        date_str = date_str.replace('/', '-')
    return date_str

# Applying the cleaning function to the invoice_date column
sales['invoice_date'] = sales['invoice_date'].apply(clean_date)

# Converting the cleaned date strings to datetime
sales['invoice_date'] = pd.to_datetime(sales['invoice_date'], dayfirst=True, errors='coerce')

In [155]:
duplicate_rows = sales.duplicated().sum()
duplicate_rows

0

In [156]:
sales.isnull().sum()

invoice_no       0
customer_id      0
category         0
quantity         5
price            3
invoice_date     0
shopping_mall    4
dtype: int64

In [157]:
#Imputing missing shopping mall values with mode as its a categorical column
sales['shopping_mall'].fillna(sales['shopping_mall'].mode()[0], inplace = True)

In [158]:
#Checking skewness
skewness = sales[['quantity', 'price']].skew()
skewness

quantity   -0.001257
price       2.247383
dtype: float64

In [159]:
#imputing null values in quantity column with mean
sales['quantity'].fillna(sales['quantity'].mean(), inplace=True)

In [160]:
#imputing null values in price column with median
sales['price'].fillna(sales['price'].median(), inplace=True)


In [161]:
#Rechecking dataframe for null values
sales.isnull().sum()

invoice_no       0
customer_id      0
category         0
quantity         0
price            0
invoice_date     0
shopping_mall    0
dtype: int64

In [162]:
#viewing unique value counts in shopping mall column
sales['shopping_mall'].value_counts()

shopping_mall
Mall of Istanbul     19945
Kanyon               19822
Metrocity            15010
Metropol AVM         10161
Istinye Park          9781
Zorlu Center          5075
Cevahir AVM           4991
Forum Istanbul        4947
Viaport Outlet        4914
Emaar Square Mall     4811
Name: count, dtype: int64

In [163]:
#Viwing unique values in category column
sales['category'].value_counts()

category
Clothing           34486
Cosmetics          15097
Food & Beverage    14775
Toys               10087
Shoes              10034
Souvenir            4998
Technology          4996
Books               4981
Clothin                1
Food & Be              1
Souvenier              1
Name: count, dtype: int64

In [164]:
category_mapping = {
    'Clothin': 'Clothing',
    'Food & Be': 'Food & Beverage',
    'Souvenier': 'Souvenir'
}

# Applying the mapping to correct the category names
sales['category'] = sales['category'].replace(category_mapping)


In [165]:
# Verifying the unique categories after correction
sales['category'].unique()

array(['Clothing', 'Shoes', 'Books', 'Cosmetics', 'Food & Beverage',
       'Toys', 'Technology', 'Souvenir'], dtype=object)

In [166]:
# Feature engineering as total sales
sales['total_sales'] = sales['quantity'] * sales['price']


In [167]:
# c.Display the initial rows of the resulting tidy datase
sales.head()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall,total_sales
0,I138884,C241288,Clothing,5.0,1500.40,2022-08-05,Kanyon,7502.00
1,I317333,C111565,Shoes,3.0,1800.51,2021-12-12,Forum Istanbul,5401.53
2,I127801,C266599,Clothing,1.0,300.08,2021-11-09,Metrocity,300.08
3,I173702,C988172,Shoes,5.0,3000.85,2021-05-16,Metropol AVM,15004.25
4,I337046,C189076,Books,4.0,60.60,2021-10-24,Kanyon,242.40


In [168]:
# d.Applying the filters
filtered_sales = sales[
    (sales['category'].isin(['Clothing', 'Shoes', 'Cosmetics'])) &
    (sales['invoice_date'] >= '2021-04-01') &
    (sales['invoice_date'] <= '2023-03-31')]
filtered_sales.head()

,invoice_no,customer_id,category,quantity,price,invoice_date,shopping_mall,total_sales
0,I138884,C241288,Clothing,5.0,1500.40,2022-08-05,Kanyon,7502.00
1,I317333,C111565,Shoes,3.0,1800.51,2021-12-12,Forum Istanbul,5401.53
2,I127801,C266599,Clothing,1.0,300.08,2021-11-09,Metrocity,300.08
3,I173702,C988172,Shoes,5.0,3000.85,2021-05-16,Metropol AVM,15004.25
5,I227836,C657758,Clothing,5.0,1500.40,2022-05-24,Forum Istanbul,7502.00


In [170]:
# Saving the cleaned filtered dataset to a CSV file
filtered_sales.to_csv('fashion&beauty_sales1.0.csv', index=False)

In [175]:
import pyodbc

# Define the connection parameters for the SQL Server
server = 'DESKTOP-VK2UTCK\IC' 
database = 'Retail'  
# Establishing the connection to the database using Windows Authentication
try:
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        'Trusted_Connection=yes;'
    )
    cursor = conn.cursor()
    print("Connection established successfully using Windows Authentication.")

except pyodbc.Error as e:
    print(f"Error connecting to SQL Server: {e}")

Connection established successfully using Windows Authentication.


In [179]:
try:
    # Create a cursor for the new database connection
    cursor_retail_sales = connection_retail_sales.cursor()

    # Define the SQL query to create a new table
    create_table_query = """
    CREATE TABLE Fashion (
        invoice_no VARCHAR(50),
        customer_id VARCHAR(50),
        category VARCHAR(50),
        quantity INT,
        price FLOAT,
        invoice_date DATE,
        shopping_mall VARCHAR(50)
    )
    """

     # Commit the table creation to ensure it is completed
    conn.commit()
    print("Table created successfully (if it didn't already exist).")

except Exception as e:
    conn.rollback()
    print(f"Error creating table: {e}")


Error creating table: name 'connection_retail_sales' is not defined


In [ ]:
try:
    # Insert data into the table
    for index, row in sales.iterrows():
        cursor.execute('''
        INSERT INTO sales (invoice_no, customer_id, category, quantity, price, invoice_date, shopping_mall, total_sales) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''')#, 
        #row['invoice_no'], row['customer_id'], row['category'], row['quantity'], row['price'], row['invoice_date'], row['shopping_mall'], row['total_sales'])

    # Commit the transaction
    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    conn.rollback()
    print(f"Error inserting data: {e}")

In [44]:
try:
    # Create a cursor for the new database connection
    cursor_retail_sales = connection_retail_sales.cursor()

    # Insert data into the SQL Server table
    for index, row in filtered_df.iterrows():
        cursor_retail_sales.execute("""
        INSERT INTO sales_data (invoice_no, customer_id, category, quantity, price, invoice_date, shopping_mall) 
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """, 
        row['invoice_no'], row['customer_id'], row['category'], row['quantity'], row['price'], row['invoice_date'], row['shopping_mall'])

    # Commit the transaction
    connection_retail_sales.commit()
    print("Data inserted successfully.")

    # Close the cursor
    cursor_retail_sales.close()
except Exception as e:
    print("Error while inserting data:", e)

Data inserted successfully.


In [45]:
try:
    # Create a cursor for the new database connection
    cursor_retail_sales = connection_retail_sales.cursor()

    # Define the SQL query to retrieve the top 50 records from the table
    select_query = "SELECT * FROM sales_data"

    # Execute the query
    cursor_retail_sales.execute(select_query)

    # Fetch all rows from the executed query
    rows = cursor_retail_sales.fetchall()

    # Fetch the column names
    columns = [column[0] for column in cursor_retail_sales.description]

    # Create a DataFrame to display the data with column names
    df = pd.DataFrame.from_records(rows, columns=columns)

    # Display the DataFrame
    print(df)

    # Close the cursor
    cursor_retail_sales.close()
except Exception as e:
    print("Error while retrieving data:", e)

      invoice_no customer_id   category  quantity    price invoice_date  \
0        I138884     C241288   Clothing         5  1500.40   2022-08-05   
1        I317333     C111565      Shoes         3  1800.51   2021-12-12   
2        I127801     C266599   Clothing         1   300.08   2021-11-09   
3        I173702     C988172      Shoes         5  3000.85   2021-05-16   
4        I227836     C657758   Clothing         5  1500.40   2022-05-24   
...          ...         ...        ...       ...      ...          ...   
52939    I662891     C180055  Cosmetics         3   121.98   2021-09-03   
52940    I887161     C137631   Clothing         1   300.08   2021-06-22   
52941    I152790     C223470  Cosmetics         4   162.64   2021-07-26   
52942    I332105     C231387      Shoes         4  2400.68   2021-08-29   
52943    I134399     C953724   Clothing         1   300.08   2023-01-01   

          shopping_mall  
0                Kanyon  
1        Forum Istanbul  
2             Metroci

In [46]:
# Save DataFrame to a CSV file
csv_file_path = 'E:\\NZSE\\612_DataTransformation\\Assignment 2\\Retail_sales.csv'
df.to_csv(csv_file_path, index=False)

In [47]:
from google.cloud import storage

# Path to your service account key file
service_account_key_path = 'E:\NZSE\612_DataTransformation\Assignment 2\playground-s-11-d23dd9a6-9e8d5e09e5c8.json'

# Name of the new bucket
bucket_name = 'Retail_Sales'

# Path to the CSV file
csv_file_path = 'E:\\NZSE\\612_DataTransformation\\Assignment 2\\Retail_sales.csv'

# Destination path in the bucket
destination_blob_name = 'Retail_sales.csv'

# Initialize a client
storage_client = storage.Client.from_service_account_json(service_account_key_path)

# Create the bucket
bucket = storage_client.create_bucket(bucket_name)
print(f"Bucket {bucket_name} created.")

# Get the bucket
bucket = storage_client.bucket(bucket_name)

# Create a blob
blob = bucket.blob(destination_blob_name)

# Upload the file
blob.upload_from_filename(csv_file_path)

print(f"File {csv_file_path} uploaded to {bucket_name}/{destination_blob_name}.")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: malformed \N character escape (2281854510.py, line 4)

In [ ]:
import os
import schedule
import time
from google.cloud import storage
from datetime import datetime

# Configuration
csv_file_path = 'path/to/your/file.csv'  # Path to your CSV file
service_account_key_path = 'path/to/your/service-account-key.json'  # Path to your service account key file
bucket_name = 'your-bucket-name'  # Your Google Cloud Storage bucket name

def upload_to_gcs(file_path, service_account_key_path, bucket_name):
    try:
        # Initialize the storage client
        storage_client = storage.Client.from_service_account_json(service_account_key_path)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(os.path.basename(file_path))
        
        # Upload the file
        blob.upload_from_filename(file_path)
        print(f'File {file_path} uploaded to {bucket_name}/{os.path.basename(file_path)}')
    except Exception as e:
        print(f'Error uploading {file_path} to GCS: {e}')

def backup_task():
    print("Starting backup process...")
    upload_to_gcs(csv_file_path, service_account_key_path, bucket_name)
    print("Backup process completed.")

# Schedule the backup task at 8:30 PM every day
schedule.every().day.at("20:30").do(backup_task)

print("Scheduler started. Waiting for the next backup time...")

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
import os
import zipfile
from datetime import datetime
from google.cloud import storage

# Configuration
directory_to_backup = 'E:\\NZSE\\612\\Assignment'
service_account_key_path = 'path/to/your/service-account-key.json'
bucket_name = 'your_new_bucket_name'

# Generate zip file name with timestamp
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
zip_filename = f'E:\\NZSE\\612\\Assignment_backup_{timestamp}.zip'

# Create a zip file
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(directory_to_backup):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=directory_to_backup)
            zipf.write(file_path, arcname)

print(f'Backup created: {zip_filename}')

# Upload to Google Cloud Storage
def upload_to_gcs(zip_filename, service_account_key_path, bucket_name):
    storage_client = storage.Client.from_service_account_json(service_account_key_path)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(os.path.basename(zip_filename))
    blob.upload_from_filename(zip_filename)
    print(f'File {zip_filename} uploaded to {bucket_name}/{os.path.basename(zip_filename)}')

upload_to_gcs(zip_filename, service_account_key_path, bucket_name)